In [2]:
import praw
from psaw import PushshiftAPI
import datetime as dt
import pandas as pd

api = PushshiftAPI()

In [3]:

start_epoch=int(dt.datetime(2021, 2, 18).timestamp())
end_epoch = int(dt.datetime(2021, 2, 21).timestamp())
subreddit = 'trans'
results = list(api.search_submissions(after=start_epoch,
                            subreddit=subreddit,
                            filter=['id', 'title', 'subreddit', 'num_comments', 'score'],
                            limit=10))

In [4]:


title_results = pd.DataFrame([thing.title for thing in results])

submission_results_df = pd.DataFrame(title_results)
fileName = "data/" + subreddit + str(start_epoch) + ".csv"
submission_results_df.to_csv(fileName.format(start_epoch, end_epoch))